<a href="https://colab.research.google.com/github/andentze/FaceColab_Unofficial/blob/main/FaceColab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Faceswap Notebook
 
Welcome to the Faceswap *not official* notebook. Ever had a problem with your PC/laptop not being good enough for Faceswap? This notebook will help you out with **everything** regarding Faceswap.
 
 
Feel free to set up the environment as you like. I, personally, prefer my dataset and the model being on the Google Drive directly, or something like this:
 
> "/content/drive/My Drive/colab/faceswap/model
 
> "/content/drive/My Drive/colab/faceswap/faces/A
 
> /content/drive/My Drive/colab/faceswap/faces/B
 
> /content/drive/My Drive/colab/faceswap/config
 
and so on.
 
You can choose to either use the GUI version of Faceswap(*which can take some time to set up*) or the CLI version of Faceswap that is present here.
 
You will also receive 4 kinds of GPUs: P4, T4, K80 or P100(*different kinds of GPU's in Colab Pro which are proven to be faster*). Also, you get 12 hours of notebook usage until you get disconnected(24 hours in Colab Pro). You *might* get frequent disconnects.
 
And before you start using this notebook, clone it to your Google Drive. (File -> Save a copy in Drive)

**I TAKE NO CREDIT FOR ANY REPOSITORIES, NOR SOFTWARE USED IN THIS NOTEBOOK. ANYTHING USED HERE IS NOT MINE.**

*If you wish to support me by buying me a donut, feel free to [donate](https://www.donationalerts.com/r/andentze).* You *may* get early access to the notebook updates, which I'm not really sure is needed. But at the end of the day, it's one way to support me.

In [ ]:
#@title Check the current GPU
#@markdown Right here you can check what kind of GPU you have available right now. Run this code to output the GPU used.
#@markdown If it errors out, make sure your runtime type is set to "GPU".

!nvidia-smi

In [ ]:
#@title Keep-Alive Script
#@markdown You have to activate this to automatically reconnect if Colab disconnects you. Run this code, to activate the "Keep-Alive" script.
#@markdown And just to make sure, open the console using Ctrl+Shift+I, and paste the code below. This will work efficently for both CLI and DE.
 
import IPython
from google.colab import output
 
display(IPython.display.Javascript('''
function ClickConnect() {
  console.log('Working')
  document
    .querySelector('#top-toolbar > colab-connect-button')
    .shadowRoot.querySelector('#connect')
    .click()
}
 
setInterval(ClickConnect, 60000)
'''))
 
print("Done.")

> function ClickConnect() {

>  console.log('Working')

>  document

>    .querySelector('#top-toolbar > colab-connect-button')

>    .shadowRoot.querySelector('#connect')

>    .click()
}

>    setInterval(ClickConnect, 60000)

# Setting up the CLI

Not a fan of the GUI or it simply doesn't work? Look no furher. From this point you will install Faceswap directly to Colab without the need of the DE.

It was set up so even a toddler could figure out how it works.

(*only pick one of the variants of installation*)

# Variant 1:

In [ ]:
#@title Install Faceswap and base dependencies

#@markdown Since Faceswap got official support of Tensorflow 2.8, running this will be enough for main functionality of Faceswap.

#@markdown This will take around a minute.

#@markdown If you wish to copy your config files from your Drive, feel free to fill these in.
from IPython.display import clear_output

train = "train.ini dir(/content/dir/train.ini)" #@param {type:"string"}
extract = "extract.ini dir(/content/dir/extract.ini)" #@param {type:"string"}
convert = "convert.ini dir(/content/dir/convert.ini)" #@param {type:"string"}

from google.colab import drive
drive.mount('/content/drive')

!git clone https://github.com/deepfakes/faceswap
clear_output()

!cp "{train}" faceswap/config/
!cp "{extract}" faceswap/config/
!cp "{convert}" faceswap/config/
clear_output()

%env FACESWAP_BACKEND = "nvidia"
!pip install -r "/content/faceswap/requirements/_requirements_base.txt"
clear_output()

print("You are good to go.")

# Variant 2:

In [ ]:
#@title Install Faceswap and Conda
#@markdown This will install Conda and restart the runtime.
#@markdown After the runtime restart, install dependencies below.
from IPython.display import clear_output

from google.colab import drive
drive.mount('/content/drive')

print("Installing Faceswap.")
!git clone https://github.com/deepfakes/faceswap
clear_output()

print("Installing Conda.")
!pip install condacolab
import condacolab as cc
cc.install()

In [ ]:
#@title Install dependencies
#@markdown As said before, install dependencies after the runtime restart.

#@markdown You can also fill these in to copy your config files.

train = "train.ini dir(/content/dir/train.ini)" #@param {type:"string"}
extract = "extract.ini dir(/content/dir/extract.ini)" #@param {type:"string"}
convert = "convert.ini dir(/content/dir/convert.ini)" #@param {type:"string"}

from IPython.display import clear_output
!cp "{train}" faceswap/config/
!cp "{extract}" faceswap/config/
!cp "{convert}" faceswap/config/
clear_output()
print("Config files copied.")

print("Installing Conda packages.")
!conda install -c conda-forge tqdm==4.64.0 \
numpy==1.21.5 \
Pillow==9.0.1 \
scikit-learn==1.0.2 \
fastcluster==1.2.6 \
matplotlib==3.2.2 \
imageio==2.9.0 \
imageio-ffmpeg==0.4.7 \
cudatoolkit=11.2 cudnn=8.1
clear_output()

print("Installing Pip packages.")
!pip install pip install "nvidia-ml-py<11.515" 
!pip install pip install "opencv-python>=4.5.5.0" 
!pip install pip install "ffmpy==0.2.3" 
!pip install pip install "psutil>=5.8.0" 
!pip install pip install "tensorflow-gpu>=2.2.0,<2.9.0"
clear_output()

print("You are good to go.")

And with that, you're good to go! You can go ahead, and use Faceswap at your own will!(*that is, until you hit a 12 hour mark.*)

**It appears Colab doesn't like long-terms calculations. So it will disconnect you at the most random times. BE AWARE OF THAT.**

# Workflow
 
Welcome to the Faceswap environment. Your first step is to extract the faces from your footage. After which you're going to have to fix your dataset. While PC/laptop users can fix that locally, mobile users will have to use the DE to do so.

In [ ]:
#@title Extraction
#@markdown Extraction is the first step for dataset creation. You **must** have good data in order for training to work better. Use this code snippet to extract your dataset.
 
#set variables start
 
input_dir = "/content/dir/to/video.mp4" #@param {type:"string"}
output_dir = "/content/dir/to/faces/folder" #@param {type:"string"}
detector = "s3fd" #@param ["cv2-dnn", "mtcnn", "s3fd"]
alignment = "fan" #@param ["cv2dnn", "fan"]
masker = "bisenet-fp" #@param ["bisenet-fp", "unet-dfl", "vgg-clear", "vgg-obstructed"]
al_normalization = "hist" #@param ["clahe", "hist", "mean"]
refeed = 4 #@param {type:"slider", min:0, max:10, step:1}
size = 512 #@param {type:"slider", min:256, max:2048, step:64}
extract_every_n = 1 #@param {type:"slider", min:1, max:30, step:1}

#set variables end
!python3 '{path}'faceswap.py extract \
-i '{input_dir}' \
-o '{output_dir}' \
-D '{detector}' \
-A '{alignment}' \
-nm '{al_normalization}' \
-M '{masker}' \
-rf '{refeed}' \
-min 20 -l 0.4 \
-sz '{size}' \
-een '{extract_every_n}' \
-si 0 -ssf -L INFO

In [ ]:
#@title Extraction with saving faces
#@markdown Extraction is the first step for dataset creation. You **must** have good data in order for training to work better. Use this code snippet to extract your dataset.

#@markdown The script above extracts each face and does not save faces to the directory. Use this code to actually save faces and tinker around with them.

#set variables start

!python3 '{path}'faceswap.py extract \
-i '{input_dir}' \
-o '{output_dir}' \
-D '{detector}' \
-A '{alignment}' \
-M '{masker}' \
-nm '{al_normalization}' \
-rf '{refeed}' \
-min 20 -l 0.4 \
-sz '{size}' \
-een '{extract_every_n}' \
-si 0 -L INFO

In [ ]:
#@title Unzip the dataset
#@markdown If you have the dataset ready to use on your PC and it's quite massive, it's *much* better to pack it to the .zip file and upload it to the Drive.

zip_directory = "/content/dir/to/file.zip" #@param {type:"string"}
output_directory = "/content/dir/to/faces/A or B" #@param {type:"string"}
#@markdown Output directory depends on which side of the face are you unpacking. Do note that it may take a while to unpack and may heavily fill your Drive storage.

#@markdown The original .zip file will be deleted right after the unpacking is finished.

from IPython.display import clear_output

!unzip '{zip_directory}' -d '{output_directory}'
!rm '{zip_directory}'
clear_output()
print("Done.")

In [ ]:
#@title Training
#@markdown Now, you have your dataset at tip-top shape! All you have to do is start training now! But wait. Before you actually start, make sure your configuration was set right.
 
#@markdown Made sure? Then get to training by setting up the variables first.
from IPython.display import clear_output

#set variables start
input_a = "/content/dir/to/faces/A" #@param {type:"string"}
input_b = "/content/dir/to/faces/B" #@param {type:"string"}
num_iterations = 100000 #@param {type:"slider", min:100000, max:3000000, step:25000}
save_every = 360 #@param {type:"slider", min:360, max:1800, step:360}
save_model_every = 25000 #@param {type:"slider", min:25000, max:100000, step:25000}
batch_num = 16 #@param {type:"slider", min:4, max:72, step:4}
trainer_type = "dfaker" #@param ["lightweight", "original", "iae", "dfaker", "dlight", "unbalanced", "dfl-h128", "villain", "phaze-a"] 
model_dir = "/content/dir/to/model" #@param {type:"string"}
timelapse_dir = "/content/dir/to/timelapse" #@param {type:"string"}

#set variables end

print("Please wait, while it pre-loads the A dataset.")
print("This may take a while depending on how many faces you have.")
print("Pre-loading is needed to load EVERY image in the directory for Faceswap.")
print("If the dataset is not pre-loaded, some images may not load into the model.")
!dir '{input_a}'
clear_output()
print("Please wait, while it pre-loads the B dataset.")
print("This may take a while depending on how many faces you have.")
print("Pre-loading is needed to load EVERY image in the directory for Faceswap.")
print("If the dataset is not pre-loaded, some images may not load into the model.")
!dir '{input_b}'
clear_output()

   #fit training args: -nw -nf
 
!python3 '{path}'/faceswap.py train \
  -A '{input_a}' \
  -B '{input_b}' \
  -m '{model_dir}' \
  -t '{trainer_type}' \
  -bs '{batch_num}' \
  -it '{num_iterations}' \
  -s '{save_every}' \
  -ss '{save_model_every}' \
  -tia '{input_a}' \
  -tib '{input_b}' \
  -to '{timelapse_dir}' \
  -w 

In [ ]:
#@title Training with No Warp
#@markdown Disabling the warp can let you achieve extra sharpness. Start this training sequence if you think your model has trained enough(*judge by the previews*).
#@markdown Your variables are saved from the training session above. If not, start your normal training sequence first, then terminate it.


!python3 '{path}'/faceswap.py train \
  -A '{input_a}' \
  -B '{input_b}' \
  -m '{model_dir}' \
  -t '{trainer_type}' \
  -bs '{batch_num}' \
  -it '{num_iterations}' \
  -s '{save_every}' \
  -ss '{save_model_every}' \
  -tia '{input_a}' \
  -tib '{input_b}' \
  -to '{timelapse_dir}' \
  -w \
  -nw

In [ ]:
#@title Convert
#@markdown After your model is trained enough, you can convert your final result. Fill the variables and run the code yet another time.

#variables set
input_video = "/content/dir/videoA.mp4" #@param {type:"string"}
output_dir = "/content/dir/output" #@param {type:"string"}
alignments = "/content/dir/alignments.fsa" #@param {type:"string"}
model_dir = "/content/dir/your_model" #@param {type:"string"}
color_adj = "" #@param ["", "avg-color", "color-transfer", "manual-balance", "match-hist", "seamless-clone"]
mask = "" #@param ["", "components", "extended", "unet-dfl", "vgg-clear", "vgg-obstructed", "predicted", "bisenet-fp_face", "bisenet-fp_head"]
writer = "ffmpeg" #@param ["ffmpeg", "gif", "opencv", "pillow"]
output_scale = 100 #@param {type:"slider", min:25, max:400, step:5}
#@markdown "Predicted" mask type is used only if you trained your own mask using "Learn Mask". If you're not sure, use "Components" or "Extended" as they are used by default.
#variables end

!python3 faceswap/faceswap.py convert \
-i '{input_video}' \
-o '{output_dir}' \
-al '{alignments}' \
-m '{model_dir}' \
-c '{color_adj}' \
-M '{mask}' \
-w '{writer}' \
-osc '{output_scale}' -l 0.4 -j 0 -L INFO

In [ ]:
#@title Convert with Swap Model
#@markdown Noticed that you trained the model the other way around? Do not worry. Run this to convert it the other way around.

#@markdown The variables should stay the same as from the code above.

!python3 faceswap/faceswap.py convert \
-i '{input_video}' \
-o '{output_dir}' \
-al '{alignments}' \
-m '{model_dir}' \
-c '{color_adj}' \
-M '{mask}aa' \
-w '{writer}' \
-osc '{output_scale}' -s -l 0.4 -j 0 -L INFO

# Various Tools

*Use if necessary.*

Faceswap provides a couple of handy tools for you to make your dataset managment easier. Here you can do whatever you want(*limited to the "Manual" tool as it requires a desktop environment*).

# Faceswap Tools

These are the most handy tools to use for managing the dataset using Faceswap.

In [ ]:
#@title Alignments
#@markdown This tool allows you to perform various actions to the alignments file. It is usually stored next to the video source file.
#@markdown You would only need "Remove Faces" job for most of the time.
#@markdown If you wish to use any other tool, proceed to the guides to find out how to use some of them.
#set variables
job = "remove-faces" #@param ["draw", "extract", "missing-alignments", "missing-frames", "multi-faces", "no-faces", "remove-faces", "rename", "sort"]
alignments = "/content/dir/alignments.fsa" #@param {type:"string"}
faces = "/content/dir/A or B" #@param {type:"string"}
drive_install = False #@param {type:"boolean"}
if drive_install:
  path = "/content/drive/My Drive/faceswap/"
  print("The installation is on Google Drive. Using Drive directory.")
if not drive_install:
  path = "/content/faceswap/"
  print("The installation is local. Using local directory.")
#end variables
!python3 faceswap\tools.py alignments \
-j '{job}' \
-o console \
-a '{alignments}' \
-fc '{faces}' \
-een 1 -sz 512 -L INFO


In [ ]:
#@title FFmpeg
#@markdown Generate videos for timelapse(*usually this tool is required for that*). **Strongly** recommended to use BRU first to rename each timestamp to start from 000000.jpeg.

#set variables
input = "your/timelapse/dir/here" #@param {type:"string"}
output = "your/video/output/here/example.mp4" #@param {type:"string"}
fps = 5 #@param {type:"slider", min:1, max:60}

#end variables
!python3 faceswap\tools.py effmpeg \
-a gen-vid \
-i input \
-o output \
-fps fps \
-ef .png \
-s 00:00:00 \
-e 00:00:00 \
-d 00:00:00 -sc 1920x1080 -L INFO

In [ ]:
#@title Sort

#@markdown This is the Sort tool. It helps you with cleaning your dataset more efficiently.

#@markdown The sorting may require **a lot** of RAM depending on your dataset quantity. The bigger your dataset is, the more RAM you may need.

#@markdown The formula for calculating the ammount of RAM is: *(n² * 24) / 1.8* where *n* is the number of images.
#@markdown The number you'll get is in bytes, so divide it by 1024 for megabytes, and by 1024 again to get gigabytes.

#@markdown If the sorting fails(*i.e. the OOM issue*), divide the dataset in some small parts and sort each one of them.

#@markdown Please note, that there are only those jobs, that **I** think are useful.
input = "/content/drive/MyDrive/colab_files/faceswap/faces/B" #@param{type:"string"}
job = "face-yaw" #@param ["face", "blur", "hist", "face-yaw"]

!python3 faceswap/tools.py sort \
-i '{input}' \
-s '{job}' \
-t -1.0 -fp rename -g hist -b 5 -lf sort_log.json -L INFO

# Config

Here you can change the config files. Only change the orange text.

**WARNING: HUGE WALL OF TEXT AHEAD**

# extract.ini

In [ ]:
config = """
[global]
# OPTIONS THAT APPLY TO ALL EXTRACTION PLUGINS

# [Nvidia Only]. Enable the Tensorflow GPU `allow_growth` configuration option. This option prevents
# Tensorflow from allocating all of the GPU VRAM at launch but can lead to higher VRAM fragmentation
# and slower performance. Should only be enabled if you are having problems running extraction.
# 
# Choose from: True, False
# [Default: False]
allow_growth = False

[align.fan]
# FAN ALIGNER OPTIONS.
# FAST ON GPU, SLOW ON CPU. BEST ALIGNER.

# The batch size to use. To a point, higher batch sizes equal better performance, but setting it too
# high can harm performance.
# 
#     - Nvidia users: If the batchsize is set higher than the your GPU can accomodate then this will
# 		automatically be lowered.
#     - AMD users: A batchsize of 8 requires about 4 GB vram.
# 
# Select an integer between 1 and 64
# [Default: 12]
batch-size = 12

[detect.cv2_dnn]
# CV2 DNN DETECTOR OPTIONS.
# A CPU ONLY EXTRACTOR, IS THE LEAST RELIABLE, BUT USES LEAST RESOURCES AND RUNS FAST ON CPU. USE THIS
# IF NOT USING A GPU AND TIME IS IMPORTANT

# The confidence level at which the detector has succesfully found a face.
# Higher levels will be more discriminating, lower levels will have more false positives.
# 
# Select an integer between 25 and 100
# [Default: 50]
confidence = 50

[detect.mtcnn]
# MTCNN DETECTOR OPTIONS.
# FAST ON GPU, SLOW ON CPU. USES FEWER RESOURCES THAN OTHER GPU DETECTORS BUT CAN OFTEN RETURN MORE
# FALSE POSITIVES.

# The minimum size of a face (in pixels) to be accepted as a positive match.
# Lower values use significantly more VRAM and will detect more false positives.
# 
# Select an integer between 20 and 1000
# [Default: 20]
minsize = 20

# The scale factor for the image pyramid.
# 
# Select a decimal number between 0.1 and 0.9
# [Default: 0.709]
scalefactor = 0.709

# The batch size to use. To a point, higher batch sizes equal better performance, but setting it too
# high can harm performance.
# 
#     - Nvidia users: If the batchsize is set higher than the your GPU can accomodate then this will
# 		automatically be lowered.
# 
# Select an integer between 1 and 64
# [Default: 8]
batch-size = 8

# First stage threshold for face detection. This stage obtains face candidates.
# 
# Select a decimal number between 0.1 and 0.9
# [Default: 0.6]
threshold_1 = 0.6

# Second stage threshold for face detection. This stage refines face candidates.
# 
# Select a decimal number between 0.1 and 0.9
# [Default: 0.7]
threshold_2 = 0.7

# Third stage threshold for face detection. This stage further refines face candidates.
# 
# Select a decimal number between 0.1 and 0.9
# [Default: 0.7]
threshold_3 = 0.7

[detect.s3fd]
# S3FD DETECTOR OPTIONS.
# FAST ON GPU, SLOW ON CPU. CAN DETECT MORE FACES AND FEWER FALSE POSITIVES THAN OTHER GPU DETECTORS,
# BUT IS A LOT MORE RESOURCE INTENSIVE.

# The confidence level at which the detector has succesfully found a face.
# Higher levels will be more discriminating, lower levels will have more false positives.
# 
# Select an integer between 25 and 100
# [Default: 70]
confidence = 70

# The batch size to use. To a point, higher batch sizes equal better performance, but setting it too
# high can harm performance.
# 
#     - Nvidia users: If the batchsize is set higher than the your GPU can accomodate then this will
# 		automatically be lowered.
#     - AMD users: A batchsize of 8 requires about 2 GB vram.
# 
# Select an integer between 1 and 64
# [Default: 4]
batch-size = 4

[mask.bisenet_fp]
# BISENET FACE PARSING OPTIONS.
# MASK PORTED FROM HTTPS://GITHUB.COM/ZLLRUNNING/FACE-PARSING.PYTORCH.

# The batch size to use. To a point, higher batch sizes equal better performance, but setting it too
# high can harm performance.
# 
#     - Nvidia users: If the batchsize is set higher than the your GPU can accomodate then this will
# 		automatically be lowered.
# 
# Select an integer between 1 and 64
# [Default: 8]
batch-size = 8

# The trained weights to use.
# 
#     - faceswap - Weights trained on wildly varied Faceswap extracted data to better handle varying
# 		conditions, obstructions, glasses and multiple targets within a single extracted image.
#     - original - The original weights trained on the CelebAMask-HQ dataset.
# 
# Choose from: ['faceswap', 'original']
# [Default: faceswap]
weights = faceswap

# Whether to include ears within the face mask.
# 
# Choose from: True, False
# [Default: False]
include_ears = False

# Whether to include hair within the face mask.
# 
# Choose from: True, False
# [Default: False]
include_hair = False

# Whether to include glasses within the face mask.
#     - For 'original' weights excluding glasses will mask out the lenses as well as the frames.
#     - For 'faceswap' weights, the model has been trained to mask out lenses if eyes cannot be seen
# 		(i.e. dark sunglasses) or just the frames if the eyes can be seen.
# 
# Choose from: True, False
# [Default: True]
include_glasses = True

[mask.unet_dfl]
# UNET_DFL OPTIONS. MASK DESIGNED TO PROVIDE SMART SEGMENTATION OF MOSTLY FRONTAL FACES.
# THE MASK MODEL HAS BEEN TRAINED BY COMMUNITY MEMBERS. INSERT MORE COMMENTARY ON TESTING HERE.
# PROFILE FACES MAY RESULT IN SUB-PAR PERFORMANCE.

# The batch size to use. To a point, higher batch sizes equal better performance, but setting it too
# high can harm performance.
# 
#     - Nvidia users: If the batchsize is set higher than the your GPU can accomodate then this will
# 		automatically be lowered.
# 
# Select an integer between 1 and 64
# [Default: 8]
batch-size = 8

[mask.vgg_clear]
# VGG_CLEAR OPTIONS. MASK DESIGNED TO PROVIDE SMART SEGMENTATION OF MOSTLY FRONTAL FACES CLEAR OF
# OBSTRUCTIONS.
# PROFILE FACES AND OBSTRUCTIONS MAY RESULT IN SUB-PAR PERFORMANCE.

# The batch size to use. To a point, higher batch sizes equal better performance, but setting it too
# high can harm performance.
# 
#     - Nvidia users: If the batchsize is set higher than the your GPU can accomodate then this will
# 		automatically be lowered.
# 
# Select an integer between 1 and 64
# [Default: 6]
batch-size = 6

[mask.vgg_obstructed]
# VGG_OBSTRUCTED OPTIONS. MASK DESIGNED TO PROVIDE SMART SEGMENTATION OF MOSTLY FRONTAL FACES.
# THE MASK MODEL HAS BEEN SPECIFICALLY TRAINED TO RECOGNIZE SOME FACIAL OBSTRUCTIONS (HANDS AND
# EYEGLASSES). PROFILE FACES MAY RESULT IN SUB-PAR PERFORMANCE.

# The batch size to use. To a point, higher batch sizes equal better performance, but setting it too
# high can harm performance.
# 
#     - Nvidia users: If the batchsize is set higher than the your GPU can accomodate then this will
# 		automatically be lowered.
# 
# Select an integer between 1 and 64
# [Default: 2]
batch-size = 2


"""

with open("faceswap/config/extract.ini", "w") as text_file:
    text_file.write(config)

# train.ini

In [ ]:
config = """
[global]
# OPTIONS THAT APPLY TO ALL MODELS
# NB: UNLESS SPECIFICALLY STATED, VALUES CHANGED HERE WILL ONLY TAKE EFFECT WHEN CREATING A NEW MODEL.

# How to center the training image. The extracted images are centered on the middle of the skull based
# on the face's estimated pose. A subsection of these images are used for training. The centering used
# dictates how this subsection will be cropped from the aligned images.
#     - face: Centers the training image on the center of the face, adjusting for pitch and yaw.
#     - head: Centers the training image on the center of the head, adjusting for pitch and yaw. NB:
# 		You should only select head centering if you intend to include the full head (including hair) in
# 		the final swap. This may give mixed results. Additionally, it is only worth choosing head
# 		centering if you are training with a mask that includes the hair (e.g. BiSeNet-FP-Head).
#     - legacy: The 'original' extraction technique. Centers the training image near the tip of the
# 		nose with no adjustment. Can result in the edges of the face appearing outside of the training
# 		area.
# 
# Choose from: ['face', 'head', 'legacy']
# [Default: face]
centering = face

# How much of the extracted image to train on. A lower coverage will limit the model's scope to a
# zoomed-in central area while higher amounts can include the entire face. A trade-off exists between
# lower amounts given more detail versus higher amounts avoiding noticeable swap transitions. For
# 'Face' centering you will want to leave this above 75%. For Head centering you will most likely want
# to set this to 100%. Sensible values for 'Legacy' centering are:
#     - 62.5% spans from eyebrow to eyebrow.
#     - 75.0% spans from temple to temple.
#     - 87.5% spans from ear to ear.
#     - 100.0% is a mugshot.
# 
# Select a decimal number between 62.5 and 100.0
# [Default: 87.5]
coverage = 87.5

# Use ICNR to tile the default initializer in a repeating pattern. This strategy is designed for
# pairing with sub-pixel / pixel shuffler to reduce the 'checkerboard effect' in image reconstruction.
#     - https://arxiv.org/ftp/arxiv/papers/1707/1707.02937.pdf
# 
# Choose from: True, False
# [Default: False]
icnr_init = False

# Use Convolution Aware Initialization for convolutional layers. This can help eradicate the vanishing
# and exploding gradient problem as well as lead to higher accuracy, lower loss and faster
# convergence.
# NB:
#     - This can use more VRAM when creating a new model so you may want to lower the batch size for
# 		the first run. The batch size can be raised again when reloading the model.
#     - Multi-GPU is not supported for this option, so you should start the model on a single GPU.
# 		Once training has started, you can stop training, enable multi-GPU and resume.
#     - Building the model will likely take several minutes as the calculations for this
# 		initialization technique are expensive. This will only impact starting a new model.
# 
# Choose from: True, False
# [Default: False]
conv_aware_init = False

# The optimizer to use.
#     - adabelief - Adapting Stepsizes by the Belief in Observed Gradients. An optimizer with the aim
# 		to converge faster, generalize better and remain more stable. (https://arxiv.org/abs/2010.07468).
# 		NB: Epsilon for AdaBelief needs to be set to a smaller value than other Optimizers. Generally
# 		setting the 'Epsilon Exponent' to around '-16' should work.
#     - adam - Adaptive Moment Optimization. A stochastic gradient descent method that is based on
# 		adaptive estimation of first-order and second-order moments.
#     - nadam - Adaptive Moment Optimization with Nesterov Momentum. Much like Adam but uses a
# 		different formula for calculating momentum.
#     - rms-prop - Root Mean Square Propagation. Maintains a moving (discounted) average of the square
# 		of the gradients. Divides the gradient by the root of this average.
# 
# Choose from: ['adabelief', 'adam', 'nadam', 'rms-prop']
# [Default: adam]
optimizer = adam

# Learning rate - how fast your network will learn (how large are the modifications to the model
# weights after one batch of training). Values that are too large might result in model crashes and
# the inability of the model to find the best solution. Values that are too small might be unable to
# escape from dead-ends and find the best global minimum.
# 
# This option can be updated for existing models.
# 
# Select a decimal number between 1e-06 and 0.0001
# [Default: 5e-05]
learning_rate = 5e-05

# The epsilon adds a small constant to weight updates to attempt to avoid 'divide by zero' errors.
# Unless you are using the AdaBelief Optimizer, then Generally this option should be left at default
# value, For AdaBelief, setting this to around '-16' should work.
# In all instances if you are getting 'NaN' loss values, and have been unable to resolve the issue any
# other way (for example, increasing batch size, or lowering learning rate), then raising the epsilon
# can lead to a more stable model. It may, however, come at the cost of slower training and a less
# accurate final result.
# NB: The value given here is the 'exponent' to the epsilon. For example, choosing '-7' will set the
# epsilon to 1e-7. Choosing '-3' will set the epsilon to 0.001 (1e-3).
# 
# This option can be updated for existing models.
# 
# Select an integer between -20 and 0
# [Default: -7]
epsilon_exponent = -7

# Use reflection padding rather than zero padding with convolutions. Each convolution must pad the
# image boundaries to maintain the proper sizing. More complex padding schemes can reduce artifacts at
# the border of the image.
#     - http://www-cs.engr.ccny.cuny.edu/~wolberg/cs470/hw/hw2_pad.txt
# 
# Choose from: True, False
# [Default: False]
reflect_padding = False

# [Nvidia Only]. Enable the Tensorflow GPU 'allow_growth' configuration option. This option prevents
# Tensorflow from allocating all of the GPU VRAM at launch but can lead to higher VRAM fragmentation
# and slower performance. Should only be enabled if you are receiving errors regarding 'cuDNN fails to
# initialize' when commencing training.
# 
# This option can be updated for existing models.
# 
# Choose from: True, False
# [Default: False]
allow_growth = False

# [Nvidia Only], NVIDIA GPUs can run operations in float16 faster than in float32. Mixed precision
# allows you to use a mix of float16 with float32, to get the performance benefits from float16 and
# the numeric stability benefits from float32.
# 
# While mixed precision will run on most Nvidia models, it will only speed up training on more recent
# GPUs. Those with compute capability 7.0 or higher will see the greatest performance benefit from
# mixed precision because they have Tensor Cores. Older GPUs offer no math performance benefit for
# using mixed precision, however memory and bandwidth savings can enable some speedups. Generally RTX
# GPUs and later will offer the most benefit.
# 
# Choose from: True, False
# [Default: False]
mixed_precision = False

# If a 'NaN' is generated in the model, this means that the model has corrupted and the model is
# likely to start deteriorating from this point on. Enabling NaN protection will stop training
# immediately in the event of a NaN. The last save will not contain the NaN, so you may still be able
# to rescue your model.
# 
# This option can be updated for existing models.
# 
# Choose from: True, False
# [Default: True]
nan_protection = True

# [GPU Only]. The number of faces to feed through the model at once when running the Convert process.
# 
# NB: Increasing this figure is unlikely to improve convert speed, however, if you are getting Out of
# Memory errors, then you may want to reduce the batch size.
# 
# This option can be updated for existing models.
# 
# Select an integer between 1 and 32
# [Default: 16]
convert_batchsize = 16

[global.loss]
# LOSS CONFIGURATION OPTIONS
# LOSS IS THE MECHANISM BY WHICH A NEURAL NETWORK JUDGES HOW WELL IT THINKS THAT IT IS RECREATING A
# FACE.
# NB: UNLESS SPECIFICALLY STATED, VALUES CHANGED HERE WILL ONLY TAKE EFFECT WHEN CREATING A NEW MODEL.

# The loss function to use.
#     - MAE - Mean absolute error will guide reconstructions of each pixel towards its median value in
# 		the training dataset. Robust to outliers but as a median, it can potentially ignore some
# 		infrequent image types in the dataset.
#     - MSE - Mean squared error will guide reconstructions of each pixel towards its average value in
# 		the training dataset. As an avg, it will be susceptible to outliers and typically produces
# 		slightly blurrier results.
#     - LogCosh - log(cosh(x)) acts similar to MSE for small errors and to MAE for large errors. Like
# 		MSE, it is very stable and prevents overshoots when errors are near zero. Like MAE, it is robust
# 		to outliers. NB: Due to a bug in PlaidML, this loss does not work on AMD cards.
#     - Smooth_L1 --- Modification of the MAE loss to correct two of its disadvantages. This loss has
# 		improved stability and guidance for small errors.
#     - L_inf_norm --- The L_inf norm will reduce the largest individual pixel error in an image. As
# 		each largest error is minimized sequentially, the overall error is improved. This loss will be
# 		extremely focused on outliers.
#     - SSIM - Structural Similarity Index Metric is a perception-based loss that considers changes in
# 		texture, luminance, contrast, and local spatial statistics of an image. Potentially delivers more
# 		realistic looking images.
#     - MS_SSIM - Multiscale Structural Similarity Index Metric is similar to SSIM except that it
# 		performs the calculations along multiple scales of the input image. NB: This loss currently does
# 		not work on AMD Cards.
#     - GMSD - Gradient Magnitude Similarity Deviation seeks to match the global standard deviation of
# 		the pixel to pixel differences between two images. Similar in approach to SSIM. NB: This loss does
# 		not currently work on AMD cards.
#     - Pixel_Gradient_Difference - Instead of minimizing the difference between the absolute value of
# 		each pixel in two reference images, compute the pixel to pixel spatial difference in each image
# 		and then minimize that difference between two images. Allows for large color shifts, but maintains
# 		the structure of the image.
# 
# Choose from: ['mae', 'mse', 'logcosh', 'smooth_loss', 'l_inf_norm', 'ssim', 'ms_ssim', 'gmsd',
# 'pixel_gradient_diff']
# [Default: ssim]
loss_function = ssim

# The loss function to use when learning a mask.
#     - MAE - Mean absolute error will guide reconstructions of each pixel towards its median value in
# 		the training dataset. Robust to outliers but as a median, it can potentially ignore some
# 		infrequent image types in the dataset.
#     - MSE - Mean squared error will guide reconstructions of each pixel towards its average value in
# 		the training dataset. As an average, it will be susceptible to outliers and typically produces
# 		slightly blurrier results.
# 
# Choose from: ['mae', 'mse']
# [Default: mse]
mask_loss_function = mse

# The amount of L2 Regularization to apply as a penalty to Structural Similarity loss functions.
# 
# NB: You should only adjust this if you know what you are doing!
# 
# L2 regularization applies a penalty term to the given Loss function. This penalty will only be
# applied if SSIM, MS-SSIM or GMSD is selected for the main loss function, otherwise it is ignored.
# 
# The value given here is as a percentage weight of the main loss function. For example:
#     - 100 - Will give equal weighting to the main loss and the penalty function.
#     - 25 - Will give the penalty function 1/4 of the weight of the main loss function.
#     - 400 - Will give the penalty function 4x as much importance as the main loss function.
#     - 0 - Disables L2 Regularization altogether.
# 
# Select an integer between 0 and 400
# [Default: 100]
l2_reg_term = 100

# The amount of priority to give to the eyes.
# 
# The value given here is as a multiplier of the main loss score. For example:
#     - 1 - The eyes will receive the same priority as the rest of the face.
#     - 10 - The eyes will be given a score 10 times higher than the rest of the face.
# 
# NB: Penalized Mask Loss must be enable to use this option.
# 
# This option can be updated for existing models.
# 
# Select an integer between 1 and 40
# [Default: 3]
eye_multiplier = 3

# The amount of priority to give to the mouth.
# 
# The value given here is as a multiplier of the main loss score. For Example:
#     - 1 - The mouth will receive the same priority as the rest of the face.
#     - 10 - The mouth will be given a score 10 times higher than the rest of the face.
# 
# NB: Penalized Mask Loss must be enable to use this option.
# 
# This option can be updated for existing models.
# 
# Select an integer between 1 and 40
# [Default: 2]
mouth_multiplier = 2

# Image loss function is weighted by mask presence. For areas of the image without the facial mask,
# reconstruction errors will be ignored while the masked face area is prioritized. May increase
# overall quality by focusing attention on the core face area.
# 
# Choose from: True, False
# [Default: True]
penalized_mask_loss = True

# The mask to be used for training. If you have selected 'Learn Mask' or 'Penalized Mask Loss' you
# must select a value other than 'none'. The required mask should have been selected as part of the
# Extract process. If it does not exist in the alignments file then it will be generated prior to
# training commencing.
#     - none: Don't use a mask.
#     - bisenet-fp-face: Relatively lightweight NN based mask that provides more refined control over
# 		the area to be masked (configurable in mask settings). Use this version of bisenet-fp if your
# 		model is trained with 'face' or 'legacy' centering.
#     - bisenet-fp-head: Relatively lightweight NN based mask that provides more refined control over
# 		the area to be masked (configurable in mask settings). Use this version of bisenet-fp if your
# 		model is trained with 'head' centering.
#     - components: Mask designed to provide facial segmentation based on the positioning of landmark
# 		locations. A convex hull is constructed around the exterior of the landmarks to create a mask.
#     - extended: Mask designed to provide facial segmentation based on the positioning of landmark
# 		locations. A convex hull is constructed around the exterior of the landmarks and the mask is
# 		extended upwards onto the forehead.
#     - vgg-clear: Mask designed to provide smart segmentation of mostly frontal faces clear of
# 		obstructions. Profile faces and obstructions may result in sub-par performance.
#     - vgg-obstructed: Mask designed to provide smart segmentation of mostly frontal faces. The mask
# 		model has been specifically trained to recognize some facial obstructions (hands and eyeglasses).
# 		Profile faces may result in sub-par performance.
#     - unet-dfl: Mask designed to provide smart segmentation of mostly frontal faces. The mask model
# 		has been trained by community members and will need testing for further description. Profile faces
# 		may result in sub-par performance.
# 
# Choose from: ['none', 'bisenet-fp_face', 'bisenet-fp_head', 'components', 'extended', 'unet-dfl',
# 'vgg-clear', 'vgg-obstructed']
# [Default: extended]
mask_type = extended

# Apply gaussian blur to the mask input. This has the effect of smoothing the edges of the mask, which
# can help with poorly calculated masks and give less of a hard edge to the predicted mask. The size
# is in pixels (calculated from a 128px mask). Set to 0 to not apply gaussian blur. This value should
# be odd, if an even number is passed in then it will be rounded to the next odd number.
# 
# Select an integer between 0 and 9
# [Default: 3]
mask_blur_kernel = 3

# Sets pixels that are near white to white and near black to black. Set to 0 for off.
# 
# Select an integer between 0 and 50
# [Default: 4]
mask_threshold = 4

# Dedicate a portion of the model to learning how to duplicate the input mask. Increases VRAM usage in
# exchange for learning a quick ability to try to replicate more complex mask models.
# 
# Choose from: True, False
# [Default: False]
learn_mask = False

[model.dfaker]
# DFAKER MODEL (ADAPTED FROM HTTPS://GITHUB.COM/DFAKER/DF)

# Resolution (in pixels) of the output image to generate on.
# BE AWARE Larger resolution will dramatically increase VRAM requirements.
# Must be 128 or 256.
# 
# Select an integer between 128 and 256
# [Default: 128]
output_size = 128

[model.dfl_h128]
# DFL H128 MODEL (ADAPTED FROM HTTPS://GITHUB.COM/IPEROV/DEEPFACELAB)

# Lower memory mode. Set to 'True' if having issues with VRAM useage.
# NB: Models with a changed lowmem mode are not compatible with each other.
# 
# Choose from: True, False
# [Default: False]
lowmem = False

[model.dfl_sae]
# DFL SAE MODEL (ADAPTED FROM HTTPS://GITHUB.COM/IPEROV/DEEPFACELAB)

# Resolution (in pixels) of the input image to train on.
# BE AWARE Larger resolution will dramatically increase VRAM requirements.
# 
# Must be divisible by 16.
# 
# Select an integer between 64 and 256
# [Default: 128]
input_size = 128

# Controls gradient clipping of the optimizer. Can prevent model corruption at the expense of VRAM.
# 
# This option can be updated for existing models.
# 
# Choose from: True, False
# [Default: True]
clipnorm = True

# Model architecture:
#     - 'df': Keeps the faces more natural.
#     - 'liae': Can help fix overly different face shapes.
# 
# Choose from: ['df', 'liae']
# [Default: df]
architecture = df

# Face information is stored in AutoEncoder dimensions. If there are not enough dimensions then
# certain facial features may not be recognized.
# Higher number of dimensions are better, but require more VRAM.
# Set to 0 to use the architecture defaults (256 for liae, 512 for df).
# 
# Select an integer between 0 and 1024
# [Default: 0]
autoencoder_dims = 0

# Encoder dimensions per channel. Higher number of encoder dimensions will help the model to recognize
# more facial features, but will require more VRAM.
# 
# Select an integer between 21 and 85
# [Default: 42]
encoder_dims = 42

# Decoder dimensions per channel. Higher number of decoder dimensions will help the model to improve
# details, but will require more VRAM.
# 
# Select an integer between 10 and 85
# [Default: 21]
decoder_dims = 21

# Multiscale decoder can help to obtain better details.
# 
# Choose from: True, False
# [Default: False]
multiscale_decoder = False

[model.dlight]
# A LIGHTWEIGHT, HIGH RESOLUTION DFAKER VARIANT (ADAPTED FROM HTTPS://GITHUB.COM/DFAKER/DF)

# Higher settings will allow learning more features such as tatoos, piercing and wrinkles.
# Strongly affects VRAM usage.
# 
# Choose from: ['lowmem', 'fair', 'best']
# [Default: best]
features = best

# Defines detail fidelity. Lower setting can appear 'rugged' while 'good' might take a longer time to
# train.
# Affects VRAM usage.
# 
# Choose from: ['fast', 'good']
# [Default: good]
details = good

# Output image resolution (in pixels).
# Be aware that larger resolution will increase VRAM requirements.
# NB: Must be either 128, 256, or 384.
# 
# Select an integer between 128 and 384
# [Default: 256]
output_size = 256

[model.original]
# ORIGINAL FACESWAP MODEL.

# Lower memory mode. Set to 'True' if having issues with VRAM useage.
# NB: Models with a changed lowmem mode are not compatible with each other.
# 
# Choose from: True, False
# [Default: False]
lowmem = False

[model.phaze_a]
# PHAZE-A MODEL BY TORZDF, WITH THANKS TO BIRBFAKES.
# ALLOWS FOR THE EXPERIMENTATION OF VARIOUS STANDARD NETWORKS AS THE ENCODER AND TAKES INSPIRATION
# FROM NVIDIA'S STYLEGAN FOR THE DECODER. IT IS HIGHLY RECOMMENDED TO RESEARCH TO UNDERSTAND THE
# PARAMETERS BETTER.

# Resolution (in pixels) of the output image to generate.
# BE AWARE Larger resolution will dramatically increase VRAM requirements.
# 
# Select an integer between 64 and 1024
# [Default: 128]
output_size = 128

# Whether to create a shared fully connected layer. This layer will have the same structure as the
# fully connected layers used for each side of the model. A shared fully connected layer looks for
# patterns that are common to both sides. NB: Enabling this option only makes sense if 'split fc' is
# selected.
#     - none - Do not create a Fully Connected layer for shared data. (Original method)
#     - full - Create an exclusive Fully Connected layer for shared data. (IAE method)
#     - half - Use the 'fc_a' layer for shared data. This saves VRAM by re-using the 'A' side's fully
# 		connected model for the shared data. However, this will lead to an 'unbalanced' model and can lead
# 		to more identity bleed (DFL method)
# 
# Choose from: ['none', 'full', 'half']
# [Default: none]
shared_fc = none

# Whether to enable the G-Block. If enabled, this will create a shared fully connected layer
# (configurable in the 'G-Block hidden layers' section) to look for patterns in the combined data,
# before feeding a block prior to the decoder for merging this shared and combined data.
#     - True - Use the G-Block in the Decoder. A combined fully connected layer will be created to
# 		feed this block which can be configured below.
#     - False - Don't use the G-Block in the decoder. No combined fully connected layer will be
# 		created.
# 
# Choose from: True, False
# [Default: True]
enable_gblock = True

# Whether to use a single shared Fully Connected layer or separate Fully Connected layers for each
# side.
#     - True - Use separate Fully Connected layers for Face A and Face B. This is more similar to the
# 		'IAE' style of model.
#     - False - Use combined Fully Connected layers for both sides. This is more similar to the
# 		original Faceswap architecture.
# 
# Choose from: True, False
# [Default: True]
split_fc = True

# If the G-Block is enabled, Whether to use a single G-Block shared between both sides, or whether to
# have a separate G-Block (one for each side). NB: The Fully Connected layer that feeds the G-Block
# will always be shared.
#     - True - Use separate G-Blocks for Face A and Face B.
#     - False - Use a combined G-Block layers for both sides.
# 
# Choose from: True, False
# [Default: False]
split_gblock = False

# Whether to use a single decoder or split decoders.
#     - True - Use a separate decoder for Face A and Face B. This is more similar to the original
# 		Faceswap architecture.
#     - False - Use a combined Decoder. This is more similar to 'IAE' style architecture.
# 
# Choose from: True, False
# [Default: False]
split_decoders = False

# The encoder architecture to use. See the relevant config sections for specific architecture
# tweaking.
# NB: For keras based pre-built models, the global initializers and padding options will be ignored
# for the selected encoder.
#     - densenet: (32px -224px). Ref: Densely Connected Convolutional Networks (2016):
# 		https://arxiv.org/abs/1608.06993?source=post_page
#     - efficientnet: [Tensorflow 2.3+ only] EfficientNet has numerous variants (B0 - B8) that
# 		increases the model width, depth and dimensional space at each step. The minimum input resolution
# 		is 32px for all variants. The maximum input resolution for each variant is: b0: 224px, b1: 240px,
# 		b2: 260px, b3: 300px, b4: 380px, b5: 456px, b6: 528px, b7 600px. Ref: Rethinking Model Scaling for
# 		Convolutional Neural Networks (2020): https://arxiv.org/abs/1905.11946
#     - efficientnet_v2: [Tensorflow 2.8+ only] EfficientNetV2 is the follow up to efficientnet. It
# 		has numerous variants (B0 - B3 and Small, Medium and Large) that increases the model width, depth
# 		and dimensional space at each step. The minimum input resolution is 32px for all variants. The
# 		maximum input resolution for each variant is: b0: 224px, b1: 240px, b2: 260px, b3: 300px, s:
# 		384px, m: 480px, l: 480px. Ref: EfficientNetV2: Smaller Models and Faster Training (2021):
# 		https://arxiv.org/abs/2104.00298
#     - fs_original: (32px - 160px). A configurable variant of the original facewap encoder. ImageNet
# 		weights cannot be loaded for this model. Additional parameters can be configured with the 'fs_enc'
# 		options. A version of this encoder is used in the following models: Original, Original (lowmem),
# 		Dfaker, DFL-H128, DFL-SAE, IAE, Lightweight.
#     - inception_resnet_v2: (75px - 299px). Ref: Inception-ResNet and the Impact of Residual
# 		Connections on Learning (2016): https://arxiv.org/abs/1602.07261
#     - inceptionV3: (75px - 299px). Ref: Rethinking the Inception Architecture for Computer Vision
# 		(2015): https://arxiv.org/abs/1512.00567
#     - mobilenet: (32px - 224px). Additional MobileNet parameters can be set with the 'mobilenet'
# 		options. Ref: MobileNets: Efficient Convolutional Neural Networks for Mobile Vision Applications
# 		(2017): https://arxiv.org/abs/1704.04861
#     - mobilenet_v2: (32px - 224px). Additional MobileNet parameters can be set with the 'mobilenet'
# 		options. Ref: MobileNetV2: Inverted Residuals and Linear Bottlenecks (2018):
# 		https://arxiv.org/abs/1801.04381
#     - mobilenet_v3: (32px - 224px). Additional MobileNet parameters can be set with the 'mobilenet'
# 		options. Ref: Searching for MobileNetV3 (2019): https://arxiv.org/pdf/1905.02244.pdf
#     - nasnet: (32px - 331px (large) or 224px (mobile)). Ref: Learning Transferable Architectures for
# 		Scalable Image Recognition (2017): https://arxiv.org/abs/1707.07012
#     - resnet: (32px - 224px). Deep Residual Learning for Image Recognition (2015):
# 		https://arxiv.org/abs/1512.03385
#     - vgg: (32px - 224px). Very Deep Convolutional Networks for Large-Scale Image Recognition
# 		(2014): https://arxiv.org/abs/1409.1556
#     - xception: (71px - 229px). Ref: Deep Learning with Depthwise Separable Convolutions (2017):
# 		https://arxiv.org/abs/1409.1556.
# 
# 
# Choose from: ['densenet121', 'densenet169', 'densenet201', 'efficientnet_b0', 'efficientnet_b1',
# 'efficientnet_b2', 'efficientnet_b3', 'efficientnet_b4', 'efficientnet_b5', 'efficientnet_b6',
# 'efficientnet_b7', 'efficientnet_v2_b0', 'efficientnet_v2_b1', 'efficientnet_v2_b2',
# 'efficientnet_v2_b3', 'efficientnet_v2_l', 'efficientnet_v2_m', 'efficientnet_v2_s', 'fs_original',
# 'inception_resnet_v2', 'inception_v3', 'mobilenet', 'mobilenet_v2', 'mobilenet_v3_large',
# 'mobilenet_v3_small', 'nasnet_large', 'nasnet_mobile', 'resnet101', 'resnet101_v2', 'resnet152',
# 'resnet152_v2', 'resnet50', 'resnet50_v2', 'vgg16', 'vgg19', 'xception']
# [Default: fs_original]
enc_architecture = fs_original

# Input scaling for the encoder. Some of the encoders have large input sizes, which often are not
# helpful for Faceswap. This setting scales the dimensional space that the encoder works in. For
# example an encoder with a maximum input size of 224px will be input an image of 112px at 50%%
# scaling. See the Architecture tooltip for the minimum and maximum sizes for each encoder.
# 
# Select an integer between 0 and 100
# [Default: 40]
enc_scaling = 40

# Load pre-trained weights trained on ImageNet data. Only available for non-Faceswap encoders (i.e.
# those not beginning with 'fs'). NB: If you use the global 'load weights' option and have selected to
# load weights from a previous model's 'encoder' or 'keras_encoder' then the weights loaded here will
# be replaced by the weights loaded from your saved model.
# 
# Choose from: True, False
# [Default: True]
enc_load_weights = True

# The type of layer to use for the bottleneck.
#     - average_pooling: Use a Global Average Pooling 2D layer for the bottleneck.
#     - dense: Use a Dense layer for the bottleneck (the traditional Faceswap method). You can set the
# 		size of the Dense layer with the 'bottleneck_size' parameter.
#     - max_pooling: Use a Global Max Pooling 2D layer for the bottleneck.
# 
# Choose from: ['average_pooling', 'dense', 'max_pooling']
# [Default: dense]
bottleneck_type = dense

# Apply a normalization layer after encoder output and prior to the bottleneck.
#     - none - Do not apply a normalization layer
#     - instance - Apply Instance Normalization
#     - layer - Apply Layer Normalization (Ba et al., 2016)
#     - rms - Apply Root Mean Squared Layer Normalization (Zhang et al., 2019). A simplified version
# 		of Layer Normalization with reduced overhead.
# 
# Choose from: ['none', 'instance', 'layer', 'rms']
# [Default: none]
bottleneck_norm = none

# If using a Dense layer for the bottleneck, then this is the number of nodes to use.
# 
# Select an integer between 128 and 4096
# [Default: 1024]
bottleneck_size = 1024

# Whether to place the bottleneck in the Encoder or to place it with the other hidden layers. Placing
# the bottleneck in the encoder means that both sides will share the same bottleneck. Placing it with
# the other fully connected layers means that each fully connected layer will each get their own
# bottleneck. This may be combined or split depending on your overall architecture configuration
# settings.
# 
# Choose from: True, False
# [Default: True]
bottleneck_in_encoder = True

# The number of consecutive Dense (fully connected) layers to include in each side's intermediate
# layer.
# 
# Select an integer between 1 and 16
# [Default: 1]
fc_depth = 1

# The number of filters to use for the initial fully connected layer. The number of nodes actually
# used is: fc_min_filters x fc_dimensions x fc_dimensions.
# NB: This value may be scaled down, depending on output resolution.
# 
# Select an integer between 16 and 5120
# [Default: 1024]
fc_min_filters = 1024

# This is the number of filters to be used in the final reshape layer at the end of the fully
# connected layers. The actual number of nodes used for the final fully connected layer is:
# fc_min_filters x fc_dimensions x fc_dimensions.
# NB: This value may be scaled down, depending on output resolution.
# 
# Select an integer between 128 and 5120
# [Default: 1024]
fc_max_filters = 1024

# The height and width dimension for the final reshape layer at the end of the fully connected layers.
# NB: The total number of nodes within the final fully connected layer will be: fc_dimensions x
# fc_dimensions x fc_max_filters.
# 
# Select an integer between 3 and 16
# [Default: 4]
fc_dimensions = 4

# The rate that the filters move from the minimum number of filters to the maximum number of filters.
# EG:
# Negative numbers will change the number of filters quicker at first and slow down each layer.
# Positive numbers will change the number of filters slower at first but then speed up each layer.
# 0.0 - This will change at a linear rate (i.e. the same number of filters will be changed at each
# layer).
# 
# Select a decimal number between -0.99 and 0.99
# [Default: -0.5]
fc_filter_slope = -0.5

# Dropout is a form of regularization that can prevent a model from over-fitting and help to keep
# neurons 'alive'. 0.5 will dropout half the connections between each fully connected layer, 0.25 will
# dropout a quarter of the connections etc. Set to 0.0 to disable.
# 
# This option can be updated for existing models.
# 
# Select a decimal number between 0.0 and 0.99
# [Default: 0.0]
fc_dropout = 0.0

# The type of dimensional upsampling to perform at the end of the fully connected layers, if upsamples
# > 0. The number of filters used for the upscale layers will be the value given in
# 'fc_upsample_filters'.
#     - upsample2d - A lightweight and VRAM friendly method. 'quick and dirty' but does not learn any
# 		parameters
#     - subpixel - Sub-pixel upscaler using depth-to-space which may require more VRAM.
#     - resize_images - Uses the Keras resize_image function to save about half as much vram as the
# 		heaviest methods.
#     - upscale_fast - Developed by Andenixa. Focusses on speed to upscale, but requires more VRAM.
#     - upscale_hybrid - Developed by Andenixa. Uses a combination of PixelShuffler and Upsampling2D
# 		to upscale, saving about 1/3rd of VRAM of the heaviest methods.
# 
# Choose from: ['resize_images', 'subpixel', 'upscale_fast', 'upscale_hybrid', 'upsample2d']
# [Default: upsample2d]
fc_upsampler = upsample2d

# Some upsampling can occur within the Fully Connected layers rather than in the Decoder to increase
# the dimensional space. Set how many upscale layers should occur within the Fully Connected layers.
# 
# Select an integer between 0 and 4
# [Default: 1]
fc_upsamples = 1

# If you have selected an upsampler which requires filters (i.e. any upsampler with the exception of
# Upsampling2D), then this is the number of filters to be used for the upsamplers within the fully
# connected layers,  NB: This value may be scaled down, depending on output resolution. Also note,
# that this figure will dictate the number of filters used for the G-Block, if selected.
# 
# Select an integer between 128 and 5120
# [Default: 512]
fc_upsample_filters = 512

# The number of consecutive Dense (fully connected) layers to include in the G-Block shared layer.
# 
# Select an integer between 1 and 16
# [Default: 3]
fc_gblock_depth = 3

# The number of nodes to use for the initial G-Block shared fully connected layer.
# 
# Select an integer between 128 and 5120
# [Default: 512]
fc_gblock_min_nodes = 512

# The number of nodes to use for the final G-Block shared fully connected layer.
# 
# Select an integer between 128 and 5120
# [Default: 512]
fc_gblock_max_nodes = 512

# The rate that the filters move from the minimum number of filters to the maximum number of filters
# for the G-Block shared layers. EG:
# Negative numbers will change the number of filters quicker at first and slow down each layer.
# Positive numbers will change the number of filters slower at first but then speed up each layer.
# 0.0 - This will change at a linear rate (i.e. the same number of filters will be changed at each
# layer).
# 
# Select a decimal number between -0.99 and 0.99
# [Default: -0.5]
fc_gblock_filter_slope = -0.5

# Dropout is a regularization technique that can prevent a model from over-fitting and help to keep
# neurons 'alive'. 0.5 will dropout half the connections between each fully connected layer, 0.25 will
# dropout a quarter of the connections etc. Set to 0.0 to disable.
# 
# This option can be updated for existing models.
# 
# Select a decimal number between 0.0 and 0.99
# [Default: 0.0]
fc_gblock_dropout = 0.0

# The method to use for the upscales within the decoder. Images are upscaled multiple times within the
# decoder as the network learns to reconstruct the face.
#     - subpixel - Sub-pixel upscaler using depth-to-space which requires more VRAM.
#     - resize_images - Uses the Keras resize_image function to save about half as much vram as the
# 		heaviest methods.
#     - upscale_fast - Developed by Andenixa. Focusses on speed to upscale, but requires more VRAM.
#     - upscale_hybrid - Developed by Andenixa. Uses a combination of PixelShuffler and Upsampling2D
# 		to upscale, saving about 1/3rd of VRAM of the heaviest methods.
# 
# Choose from: ['subpixel', 'resize_images', 'upscale_fast', 'upscale_hybrid']
# [Default: subpixel]
dec_upscale_method = subpixel

# Normalization to apply to apply after each upscale.
#     - none - Do not apply a normalization layer
#     - batch - Apply Batch Normalization
#     - group - Apply Group Normalization
#     - instance - Apply Instance Normalization
#     - layer - Apply Layer Normalization (Ba et al., 2016)
#     - rms - Apply Root Mean Squared Layer Normalization (Zhang et al., 2019). A simplified version
# 		of Layer Normalization with reduced overhead.
# 
# Choose from: ['none', 'batch', 'group', 'instance', 'layer', 'rms']
# [Default: none]
dec_norm = none

# The minimum number of filters to use in decoder upscalers (i.e. the number of filters to use for the
# final upscale layer).
# 
# Select an integer between 64 and 512
# [Default: 64]
dec_min_filters = 64

# The maximum number of filters to use in decoder upscalers (i.e. the number of filters to use for the
# first upscale layer).
# 
# Select an integer between 256 and 5120
# [Default: 512]
dec_max_filters = 512

# The rate that the filters reduce at each upscale layer. EG:
# Negative numbers will drop the number of filters quicker at first and slow down each upscale.
# Positive numbers will drop the number of filters slower at first but then speed up each upscale.
# 0.0 - This will reduce at a linear rate (i.e. the same number of filters will be reduced at each
# upscale).
# 
# Select a decimal number between -0.99 and 0.99
# [Default: -0.45]
dec_filter_slope = -0.45

# The number of Residual Blocks to apply to each upscale layer. Set to 0 to disable residual blocks
# entirely.
# 
# Select an integer between 0 and 8
# [Default: 1]
dec_res_blocks = 1

# The kernel size to apply to the final Convolution layer.
# 
# Select an integer between 1 and 9
# [Default: 5]
dec_output_kernel = 5

# Gaussian Noise acts as a regularization technique for preventing overfitting of data.
#     - True - Apply a Gaussian Noise layer to each upscale.
#     - False - Don't apply a Gaussian Noise layer to each upscale.
# 
# Choose from: True, False
# [Default: True]
dec_gaussian = True

# If Residual blocks have been enabled, enabling this option will not apply a Residual block to the
# final upscaler.
#     - True - Don't apply a Residual block to the final upscale.
#     - False - Apply a Residual block to all upscale layers.
# 
# Choose from: True, False
# [Default: True]
dec_skip_last_residual = True

# If the command line option 'freeze-weights' is enabled, then the layers indicated here will be
# frozen the next time the model starts up. NB: Not all architectures contain all of the layers listed
# here, so any layers marked for freezing that are not within your chosen architecture will be
# ignored. EG:
#  If 'split fc' has been selected, then 'fc_a' and 'fc_b' are available for freezing. If it has not
# been selected then 'fc_both' is available for freezing.
# 
# This option can be updated for existing models.
# 
# If selecting multiple options then each option should be separated by a space or a comma (e.g.
# item1, item2, item3)
# 
# Choose from: ['encoder', 'keras_encoder', 'fc_a', 'fc_b', 'fc_both', 'fc_shared', 'fc_gblock',
# 'g_block_a', 'g_block_b', 'g_block_both', 'decoder_a', 'decoder_b', 'decoder_both']
# [Default: keras_encoder]
freeze_layers = keras_encoder

# If the command line option 'load-weights' is populated, then the layers indicated here will be
# loaded from the given weights file if starting a new model. NB Not all architectures contain all of
# the layers listed here, so any layers marked for loading that are not within your chosen
# architecture will be ignored. EG:
#  If 'split fc' has been selected, then 'fc_a' and 'fc_b' are available for loading. If it has not
# been selected then 'fc_both' is available for loading.
# 
# If selecting multiple options then each option should be separated by a space or a comma (e.g.
# item1, item2, item3)
# 
# Choose from: ['encoder', 'fc_a', 'fc_b', 'fc_both', 'fc_shared', 'fc_gblock', 'g_block_a',
# 'g_block_b', 'g_block_both', 'decoder_a', 'decoder_b', 'decoder_both']
# [Default: encoder]
load_layers = encoder

# Faceswap Encoder only: The number of convolutions to perform within the encoder.
# 
# Select an integer between 2 and 10
# [Default: 4]
fs_original_depth = 4

# Faceswap Encoder only: The minumum number of filters to use for encoder convolutions. (i.e. the
# number of filters to use for the first encoder layer).
# 
# Select an integer between 64 and 2048
# [Default: 128]
fs_original_min_filters = 128

# Faceswap Encoder only: The maximum number of filters to use for encoder convolutions. (i.e. the
# number of filters to use for the final encoder layer).
# 
# Select an integer between 256 and 8192
# [Default: 1024]
fs_original_max_filters = 1024

# The width multiplier for mobilenet encoders. Controls the width of the network. Values less than 1.0
# proportionally decrease the number of filters within each layer. Values greater than 1.0
# proportionally increase the number of filters within each layer. 1.0 is the default number of layers
# used within the paper.
# NB: This option is ignored for any non-mobilenet encoders.
# NB: If loading ImageNet weights, then for MobilenetV1 only values of '0.25', '0.5', '0.75' or '1.0
# can be selected. For MobilenetV2 only values of '0.35', '0.50', '0.75', '1.0', '1.3' or '1.4' can be
# selected. For mobilenet_v3 only values of '0.75' or '1.0' can be selected
# 
# Select a decimal number between 0.1 and 2.0
# [Default: 1.0]
mobilenet_width = 1.0

# The depth multiplier for MobilenetV1 encoder. This is the depth multiplier for depthwise convolution
# (known as the resolution multiplier within the original paper).
# NB: This option is only used for MobilenetV1 and is ignored for all other encoders.
# NB: If loading ImageNet weights, this must be set to 1.
# 
# Select an integer between 1 and 10
# [Default: 1]
mobilenet_depth = 1

# The dropout rate for MobilenetV1 encoder.
# NB: This option is only used for MobilenetV1 and is ignored for all other encoders.
# 
# Select a decimal number between 0.001 and 2.0
# [Default: 0.001]
mobilenet_dropout = 0.001

# Use a minimilist version of MobilenetV3.
# In addition to large and small models MobilenetV3 also contains so-called minimalistic models, these
# models have the same per-layer dimensions characteristic as MobilenetV3 however, they don't utilize
# any of the advanced blocks (squeeze-and-excite units, hard-swish, and 5x5 convolutions). While these
# models are less efficient on CPU, they are much more performant on GPU/DSP.
# NB: This option is only used for MobilenetV3 and is ignored for all other encoders.
# 
# 
# Choose from: True, False
# [Default: False]
mobilenet_minimalistic = False

[model.realface]
# AN EXTRA DETAILED VARIANT OF ORIGINAL MODEL.
# INCORPORATES IDEAS FROM BRYANLYON AND INSPIRATION FROM THE VILLAIN MODEL.
# REQUIRES ABOUT 6GB-8GB OF VRAM (BATCHSIZE 8-16).
# 

# Resolution (in pixels) of the input image to train on.
# BE AWARE Larger resolution will dramatically increase VRAM requirements.
# Higher resolutions may increase prediction accuracy, but does not effect the resulting output size.
# Must be between 64 and 128 and be divisible by 16.
# 
# Select an integer between 64 and 128
# [Default: 64]
input_size = 64

# Output image resolution (in pixels).
# Be aware that larger resolution will increase VRAM requirements.
# NB: Must be between 64 and 256 and be divisible by 16.
# 
# Select an integer between 64 and 256
# [Default: 128]
output_size = 128

# Number of nodes for decoder. Might affect your model's ability to learn in general.
# Note that: Lower values will affect the ability to predict details.
# 
# Select an integer between 768 and 2048
# [Default: 1536]
dense_nodes = 1536

# Encoder Convolution Layer Complexity. sensible ranges: 128 to 150.
# 
# Select an integer between 96 and 160
# [Default: 128]
complexity_encoder = 128

# Decoder Complexity.
# 
# Select an integer between 512 and 544
# [Default: 512]
complexity_decoder = 512

[model.unbalanced]
# AN UNBALANCED MODEL WITH ADJUSTABLE INPUT SIZE OPTIONS.
# THIS IS AN UNBALANCED MODEL SO B>A SWAPS MAY NOT WORK WELL
# 

# Resolution (in pixels) of the image to train on.
# BE AWARE Larger resolution will dramatically increaseVRAM requirements.
# Make sure your resolution is divisible by 64 (e.g. 64, 128, 256 etc.).
# NB: Your faceset must be at least 1.6x larger than your required input size.
# (e.g. 160 is the maximum input size for a 256x256 faceset).
# 
# Select an integer between 64 and 512
# [Default: 128]
input_size = 128

# Lower memory mode. Set to 'True' if having issues with VRAM useage.
# NB: Models with a changed lowmem mode are not compatible with each other.
# NB: lowmem will override cutom nodes and complexity settings.
# 
# Choose from: True, False
# [Default: False]
lowmem = False

# Controls gradient clipping of the optimizer. Can prevent model corruption at the expense of VRAM.
# 
# Choose from: True, False
# [Default: True]
clipnorm = True

# Number of nodes for decoder. Don't change this unless you know what you are doing!
# 
# Select an integer between 512 and 4096
# [Default: 1024]
nodes = 1024

# Encoder Convolution Layer Complexity. sensible ranges: 128 to 160.
# 
# Select an integer between 64 and 1024
# [Default: 128]
complexity_encoder = 128

# Decoder A Complexity.
# 
# Select an integer between 64 and 1024
# [Default: 384]
complexity_decoder_a = 384

# Decoder B Complexity.
# 
# Select an integer between 64 and 1024
# [Default: 512]
complexity_decoder_b = 512

[model.villain]
# A HIGHER RESOLUTION VERSION OF THE ORIGINAL MODEL BY VILLAINGUY.
# EXTREMELY VRAM HEAVY. DON'T TRY TO RUN THIS IF YOU HAVE A SMALL GPU.
# 

# Lower memory mode. Set to 'True' if having issues with VRAM useage.
# NB: Models with a changed lowmem mode are not compatible with each other.
# 
# Choose from: True, False
# [Default: False]
lowmem = False

[trainer.original]
# ORIGINAL TRAINER OPTIONS.
# WARNING: THE DEFAULTS FOR AUGMENTATION WILL BE FINE FOR 99.9% OF USE CASES. ONLY CHANGE THEM IF YOU
# ABSOLUTELY KNOW WHAT YOU ARE DOING!

# Number of sample faces to display for each side in the preview when training.
# 
# Select an integer between 2 and 16
# [Default: 14]
preview_images = 14

# Percentage amount to randomly zoom each training image in and out.
# 
# Select an integer between 0 and 25
# [Default: 5]
zoom_amount = 5

# Percentage amount to randomly rotate each training image.
# 
# Select an integer between 0 and 25
# [Default: 10]
rotation_range = 10

# Percentage amount to randomly shift each training image horizontally and vertically.
# 
# Select an integer between 0 and 25
# [Default: 5]
shift_range = 5

# Percentage chance to randomly flip each training image horizontally.
# NB: This is ignored if the 'no-flip' option is enabled
# 
# Select an integer between 0 and 75
# [Default: 50]
flip_chance = 50

# Percentage amount to randomly alter the lightness of each training image.
# NB: This is ignored if the 'no-flip' option is enabled
# 
# Select an integer between 0 and 75
# [Default: 30]
color_lightness = 30

# Percentage amount to randomly alter the 'a' and 'b' colors of the L*a*b* color space of each
# training image.
# NB: This is ignored if the 'no-flip' option is enabled
# 
# Select an integer between 0 and 50
# [Default: 8]
color_ab = 8

# Percentage chance to perform Contrast Limited Adaptive Histogram Equalization on each training
# image.
# NB: This is ignored if the 'no-augment-color' option is enabled
# 
# This option can be updated for existing models.
# 
# Select an integer between 0 and 75
# [Default: 50]
color_clahe_chance = 50

# The grid size dictates how much Contrast Limited Adaptive Histogram Equalization is performed on any
# training image selected for clahe. Contrast will be applied randomly with a gridsize of 0 up to the
# maximum. This value is a multiplier calculated from the training image size.
# NB: This is ignored if the 'no-augment-color' option is enabled
# 
# Select an integer between 1 and 8
# [Default: 4]
color_clahe_max_size = 4
"""

with open("faceswap/config/train.ini", "w") as text_file:
    text_file.write(config)

# convert.ini

In [ ]:
config = """
[color.color_transfer]
# OPTIONS FOR TRANSFERING THE COLOR DISTRIBUTION FROM THE SOURCE TO THE TARGET IMAGE USING THE MEAN
# AND STANDARD DEVIATIONS OF THE L*A*B* COLOR SPACE.
# THIS IMPLEMENTATION IS (LOOSELY) BASED ON THE 'COLOR TRANSFER BETWEEN IMAGES' PAPER BY REINHARD ET
# AL., 2001. MATCHING THE HISTOGRAMS BETWEEN THE SOURCE AND DESTINATION FACES.

# Should components of L*a*b* image be scaled by np.clip before converting back to BGR color space?
# If False then components will be min-max scaled appropriately.
# Clipping will keep target image brightness truer to the input.
# Scaling will adjust image brightness to avoid washed out portions in the resulting color transfer
# that can be caused by clipping.
# 
# Choose from: True, False
# [Default: True]
clip = True

# Should color transfer strictly follow methodology layed out in original paper?
# The method does not always produce aesthetically pleasing results.
# If False then L*a*b* components will be scaled using the reciprocal of the scaling factor proposed
# in the paper. This method seems to produce more consistently aesthetically pleasing results.
# 
# Choose from: True, False
# [Default: True]
preserve_paper = True

[color.manual_balance]
# OPTIONS FOR MANUALLY ALTERING THE BALANCE OF COLORS OF THE SWAPPED FACE

# The colorspace to use for adjustment: The three adjustment sliders will effect the image differently
# depending on which colorspace is selected:
#     - RGB: Red, Green, Blue. An additive colorspace where colors are obtained by a linear
# 		combination of Red, Green, and Blue values. The three channels are correlated by the amount of
# 		light hitting the surface. In RGB color space the color information is separated into three
# 		channels but the same three channels also encode brightness information.
#     - HSV: Hue, Saturation, Value. Hue - Dominant wavelength. Saturation - Purity / shades of color.
# 		Value - Intensity. Best thing is that it uses only one channel to describe color (H), making it
# 		very intuitive to specify color.
#     - LAB: Lightness, A, B. Lightness - Intensity. A - Color range from green to magenta. B - Color
# 		range from blue to yellow. The L channel is independent of color information and encodes
# 		brightness only. The other two channels encode color.
#     - YCrCb: Y - Luminance or Luma component obtained from RGB after gamma correction. Cr - how far
# 		is the red component from Luma. Cb - how far is the blue component from Luma. Separates the
# 		luminance and chrominance components into different channels.
# 
# Choose from: ['RGB', 'HSV', 'LAB', 'YCrCb']
# [Default: HSV]
colorspace = HSV

# Balance of channel 1:
#     - RGB: Red
#     - HSV: Hue
#     - LAB: Lightness
#     - YCrCb: Luma
# 
# Select a decimal number between -100.0 and 100.0
# [Default: 0.0]
balance_1 = 0.0

# Balance of channel 2:
#     - RGB: Green
#     - HSV: Saturation
#     - LAB: Green > Magenta
#     - YCrCb: Distance of red from Luma
# 
# Select a decimal number between -100.0 and 100.0
# [Default: 0.0]
balance_2 = 0.0

# Balance of channel 3:
#     - RGB: Blue
#     - HSV: Intensity
#     - LAB: Blue > Yellow
#     - YCrCb: Distance of blue from Luma
# 
# Select a decimal number between -100.0 and 100.0
# [Default: 0.0]
balance_3 = 0.0

# Amount of contrast applied.
# 
# Select a decimal number between -100.0 and 100.0
# [Default: 0.0]
contrast = 0.0

# Amount of brighness applied.
# 
# Select a decimal number between -100.0 and 100.0
# [Default: 0.0]
brightness = 0.0

[color.match_hist]
# OPTIONS FOR MATCHING THE HISTOGRAMS BETWEEN THE SOURCE AND DESTINATION FACES

# Adjust the threshold for histogram matching. Can reduce extreme colors leaking in by filtering out
# colors at the extreme ends of the histogram spectrum.
# 
# Select a decimal number between 90.0 and 100.0
# [Default: 99.0]
threshold = 99.0

[mask.box_blend]
# OPTIONS FOR BLENDING THE EDGES OF THE SWAPPED BOX WITH THE BACKGROUND IMAGE

# The type of blending to use:
#     - gaussian: Blend with Gaussian filter. Slower, but often better than Normalized
#     - normalized: Blend with Normalized box filter. Faster than Gaussian
#     - none: Don't perform blending
# 
# Choose from: ['gaussian', 'normalized', 'none']
# [Default: gaussian]
type = gaussian

# The distance from the edges of the swap box to start blending.
# The distance is set as percentage of the swap box size to give the number of pixels from the edge of
# the box. Eg: For a swap area of 256px and a percentage of 4%, blending would commence 10 pixels from
# the edge.
# Higher percentages start the blending from closer to the center of the face, so will reveal more of
# the source face.
# 
# Select a decimal number between 0.1 and 25.0
# [Default: 11.0]
distance = 11.0

# Radius dictates how much blending should occur, or more specifically, how far the blending will
# spread away from the 'distance' parameter.
# This figure is set as a percentage of the swap box size to give the radius in pixels. Eg: For a swap
# area of 256px and a percentage of 5%, the radius would be 13 pixels.
# NB: Higher percentage means more blending, but too high may reveal more of the source face, or lead
# to hard lines at the border.
# 
# Select a decimal number between 0.1 and 25.0
# [Default: 5.0]
radius = 5.0

# The number of passes to perform. Additional passes of the blending algorithm can improve smoothing
# at a time cost. This is more useful for 'box' type blending.
# Additional passes have exponentially less effect so it's not worth setting this too high.
# 
# Select an integer between 1 and 8
# [Default: 1]
passes = 1

[mask.mask_blend]
# OPTIONS FOR BLENDING THE EDGES BETWEEN THE MASK AND THE BACKGROUND IMAGE

# The type of blending to use:
#     - gaussian: Blend with Gaussian filter. Slower, but often better than Normalized
#     - normalized: Blend with Normalized box filter. Faster than Gaussian
#     - none: Don't perform blending
# 
# Choose from: ['gaussian', 'normalized', 'none']
# [Default: normalized]
type = normalized

# The kernel size dictates how much blending should occur.
# The size is the diameter of the kernel in pixels (calculated from a 128px mask). This value should
# be odd, if an even number is passed in then it will be rounded to the next odd number. Higher sizes
# means more blending.
# 
# Select an integer between 1 and 9
# [Default: 3]
kernel_size = 3

# The number of passes to perform. Additional passes of the blending algorithm can improve smoothing
# at a time cost. This is more useful for 'box' type blending.
# Additional passes have exponentially less effect so it's not worth setting this too high.
# 
# Select an integer between 1 and 8
# [Default: 4]
passes = 4

# Sets pixels that are near white to white and near black to black. Set to 0 for off.
# 
# Select an integer between 0 and 50
# [Default: 4]
threshold = 4

# Erosion kernel size as a percentage of the mask radius area.
# Positive values apply erosion which reduces the size of the swapped area.
# Negative values apply dilation which increases the swapped area.
# 
# Select a decimal number between -100.0 and 100.0
# [Default: 0.0]
erosion = 0.0

[scaling.sharpen]
# OPTIONS FOR SHARPENING THE FACE AFTER PLACEMENT

# The type of sharpening to use:
#     - none: Don't perform any sharpening.
#     - box: Fastest, but weakest method. Uses a box filter to assess edges.
#     - gaussian: Slower, but better than box. Uses a gaussian filter to assess edges.
#     - unsharp-mask: Slowest, but most tweakable. Uses the unsharp-mask method to assess edges.
# 
# Choose from: ['none', 'box', 'gaussian', 'unsharp_mask']
# [Default: none]
method = none

# Percentage that controls the magnitude of each overshoot (how much darker and how much lighter the
# edge borders become).
# This can also be thought of as how much contrast is added at the edges. It does not affect the width
# of the edge rims.
# 
# Select an integer between 100 and 500
# [Default: 150]
amount = 150

# Affects the size of the edges to be enhanced or how wide the edge rims become, so a smaller radius
# enhances smaller-scale detail.
# Radius is set as a percentage of the final frame width and rounded to the nearest pixel. E.g for a
# 1280 width frame, a 0.6 percenatage will give a radius of 8px.
# Higher radius values can cause halos at the edges, a detectable faint light rim around objects. Fine
# detail needs a smaller radius.
# Radius and amount interact; reducing one allows more of the other.
# 
# Select a decimal number between 0.1 and 5.0
# [Default: 0.3]
radius = 0.3

# [unsharp_mask only] Controls the minimal brightness change that will be sharpened or how far apart
# adjacent tonal values have to be before the filter does anything.
# This lack of action is important to prevent smooth areas from becoming speckled. The threshold
# setting can be used to sharpen more pronounced edges, while leaving subtler edges untouched.
# Low values should sharpen more because fewer areas are excluded.
# Higher threshold values exclude areas of lower contrast.
# 
# Select a decimal number between 1.0 and 10.0
# [Default: 5.0]
threshold = 5.0

[writer.ffmpeg]
# OPTIONS FOR ENCODING CONVERTED FRAMES TO VIDEO.

# Video container to use.
# 
# Choose from: ['avi', 'flv', 'mkv', 'mov', 'mp4', 'mpeg', 'webm']
# [Default: mp4]
container = mp4

# Video codec to use:
#     - libx264: H.264. A widely supported and commonly used codec.
#     - libx265: H.265 / HEVC video encoder application library.
# 
# Choose from: ['libx264', 'libx265']
# [Default: libx264]
codec = libx264

# Constant Rate Factor:  0 is lossless and 51 is worst quality possible. A lower value generally leads
# to higher quality, and a subjectively sane range is 17-28. Consider 17 or 18 to be visually lossless
# or nearly so; it should look the same or nearly the same as the input but it isn't technically
# lossless.
# The range is exponential, so increasing the CRF value +6 results in roughly half the bitrate / file
# size, while -6 leads to roughly twice the bitrate.
# 
# Select an integer between 0 and 51
# [Default: 23]
crf = 23

# A preset is a collection of options that will provide a certain encoding speed to compression ratio.
# A slower preset will provide better compression (compression is quality per filesize).
# Use the slowest preset that you have patience for.
# 
# Choose from: ['ultrafast', 'superfast', 'veryfast', 'faster', 'fast', 'medium', 'slow', 'slower',
# 'veryslow']
# [Default: medium]
preset = medium

# Change settings based upon the specifics of your input:
#     - none: Don't perform any additional tuning.
#     - film: [H.264 only] Use for high quality movie content; lowers deblocking.
#     - animation: [H.264 only] Good for cartoons; uses higher deblocking and more reference frames.
#     - grain: Preserves the grain structure in old, grainy film material.
#     - stillimage: [H.264 only] Good for slideshow-like content.
#     - fastdecode: Allows faster decoding by disabling certain filters.
#     - zerolatency: Good for fast encoding and low-latency streaming.
# 
# Choose from: ['none', 'film', 'animation', 'grain', 'stillimage', 'fastdecode', 'zerolatency']
# [Default: none]
tune = none

# [H.264 Only] Limit the output to a specific H.264 profile. Don't change this unless your target
# device only supports a certain profile.
# 
# Choose from: ['auto', 'baseline', 'main', 'high', 'high10', 'high422', 'high444']
# [Default: auto]
profile = auto

# [H.264 Only] Set the encoder level, Don't change this unless your target device only supports a
# certain level.
# 
# Choose from: ['auto', '1', '1b', '1.1', '1.2', '1.3', '2', '2.1', '2.2', '3', '3.1', '3.2', '4',
# '4.1', '4.2', '5', '5.1', '5.2', '6', '6.1', '6.2']
# [Default: auto]
level = auto

# Skip muxing audio to the final video output. This will result in a video without an audio track.
# 
# Choose from: True, False
# [Default: False]
skip_mux = False

[writer.gif]
# OPTIONS FOR OUTPUTTING CONVERTED FRAMES TO AN ANIMATED GIF.

# Frames per Second.
# 
# Select an integer between 1 and 60
# [Default: 25]
fps = 25

# The number of iterations. Set to 0 to loop indefinitely.
# 
# Select an integer between 0 and 100
# [Default: 0]
loop = 0

# The number of colors to quantize the image to. Is rounded to the nearest power of two.
# 
# Choose from: ['2', '4', '8', '16', '32', '64', '128', '256']
# [Default: 256]
palettesize = 256

# If True, will try and optimize the GIF by storing only the rectangular parts of each frame that
# change with respect to the previous.
# 
# Choose from: True, False
# [Default: False]
subrectangles = False

[writer.opencv]
# OPTIONS FOR OUTPUTTING CONVERTED FRAMES TO A SERIES OF IMAGES USING OPENCV
# OPENCV CAN BE FASTER THAN OTHER IMAGE WRITERS, BUT LACKS SOME CONFIGURATION OPTIONS AND FORMATS.

# Image format to use:
#     - bmp: Windows bitmap
#     - jpg: JPEG format
#     - jp2: JPEG 2000 format
#     - png: Portable Network Graphics
#     - ppm: Portable Pixmap Format
# 
# Choose from: ['bmp', 'jpg', 'jp2', 'png', 'ppm']
# [Default: png]
format = png

# Place the swapped face on a transparent layer rather than the original frame.
# NB: This is only compatible with images saved in png format. If an incompatible format is selected
# then the image will be saved as a png.
# 
# Choose from: True, False
# [Default: False]
draw_transparent = False

# [jpg only] Set the jpg quality. 1 is worst 95 is best. Higher quality leads to larger file sizes.
# 
# Select an integer between 1 and 95
# [Default: 75]
jpg_quality = 75

# [png only] ZLIB compression level, 1 gives best speed, 9 gives best compression, 0 gives no
# compression at all.
# 
# Select an integer between 0 and 9
# [Default: 3]
png_compress_level = 3

[writer.pillow]
# OPTIONS FOR OUTPUTTING CONVERTED FRAMES TO A SERIES OF IMAGES USING PILLOW
# PILLOW IS MORE FEATURE RICH THAN OPENCV BUT CAN BE SLOWER.

# Image format to use:
#     - bmp: Windows bitmap
#     - gif: Graphics Interchange Format (NB: Not animated)
#     - jpg: JPEG format
#     - jp2: JPEG 2000 format
#     - png: Portable Network Graphics
#     - ppm: Portable Pixmap Format
#     - tif: Tag Image File Format
# 
# Choose from: ['bmp', 'gif', 'jpg', 'jp2', 'png', 'ppm', 'tif']
# [Default: png]
format = png

# Place the swapped face on a transparent layer rather than the original frame.
# NB: This is only compatible with images saved in png or tif format. If an incompatible format is
# selected then the image will be saved as a png.
# 
# Choose from: True, False
# [Default: False]
draw_transparent = False

# [gif, jpg and png only] If enabled, indicates that the encoder should make an extra pass over the
# image in order to select optimal encoder settings.
# 
# Choose from: True, False
# [Default: False]
optimize = False

# [gif only] Set whether to save the gif as interlaced or not.
# 
# Choose from: True, False
# [Default: True]
gif_interlace = True

# [jpg only] Set the jpg quality. 1 is worst 95 is best. Higher quality leads to larger file sizes.
# 
# Select an integer between 1 and 95
# [Default: 75]
jpg_quality = 75

# [png only] ZLIB compression level, 1 gives best speed, 9 gives best compression, 0 gives no
# compression at all. When optimize option is set to True this has no effect (it is set to 9
# regardless of a value passed).
# 
# Select an integer between 0 and 9
# [Default: 3]
png_compress_level = 3

# [tif only] The desired compression method for the file.
# 
# Choose from: ['none', 'tiff_ccitt', 'group3', 'group4', 'tiff_jpeg', 'tiff_adobe_deflate',
# 'tiff_thunderscan', 'tiff_deflate', 'tiff_sgilog', 'tiff_sgilog24', 'tiff_raw_16']
# [Default: tiff_deflate]
tif_compression = tiff_deflate

"""

with open("faceswap/config/convert.ini", "w") as text_file:
    text_file.write(config)